In [1]:
import pandas as pd
import numpy as np
import nltk
stop=nltk.corpus.stopwords.words('english')
from nltk import word_tokenize
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [3]:
ht_small=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/small.csv')
ht_small=ht_small.dropna()
ht_big=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/big.csv')
ht_big=ht_big.dropna()
mt_first=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp1/MBart/MBart_KO1.csv')
mt_first=mt_first.dropna()
mt_second=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp1/MBart/MBart_KO2.csv')
mt_second=mt_second.dropna()
mt_third=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp1/MBart/MBart_KO3.csv')
mt_third=mt_third.dropna()

In [4]:
small_first=[ht_small.sample(ht_small.shape[0]), mt_first.sample(mt_first.shape[0])]
small_first=pd.concat(small_first)
small_second=[ht_small.sample(ht_small.shape[0]), mt_second.sample(mt_second.shape[0])]
small_second=pd.concat(small_second)
small_third=[ht_small.sample(ht_small.shape[0]), mt_third.sample(mt_third.shape[0])]
small_third=pd.concat(small_third)
big_first=[ht_big.sample(ht_big.shape[0]), mt_first.sample(mt_first.shape[0])]
big_first=pd.concat(big_first)
big_second=[ht_big.sample(ht_big.shape[0]), mt_second.sample(mt_second.shape[0])]
big_second=pd.concat(big_second)
big_third=[ht_big.sample(ht_big.shape[0]), mt_third.sample(mt_third.shape[0])]
big_third=pd.concat(big_third)

In [5]:
def convert_label(data:pd.DataFrame):
    data['label']=data['label'].replace({data['label'].groupby(data['label']).count().keys()[0]:0, data['label'].groupby(data['label']).count().keys()[1]:1})
    return data

In [6]:
small_first=convert_label(small_first)
small_second=convert_label(small_second)
small_third=convert_label(small_third)
big_first=convert_label(big_first)
big_second=convert_label(big_second)
big_third=convert_label(big_third)

In [7]:
%%time
X=list(big_third['text'])
y=list(big_third['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

CPU times: user 1.49 ms, sys: 1.14 ms, total: 2.63 ms
Wall time: 1.75 ms


In [8]:
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=2)#, stop_words='korean')

In [9]:
X_train_count_vec = unigram_count_vectorizer.fit_transform(X_train)
X_test_count_vec = unigram_count_vectorizer.transform(X_test)

In [10]:
clf = MultinomialNB().fit(X_train_count_vec, y_train)
svm = SVC(kernel='linear', C=1, random_state=0).fit(X_train_count_vec, y_train)
tree = DecisionTreeClassifier(random_state=0).fit(X_train_count_vec, y_train)
regression = LogisticRegression(random_state=0).fit(X_train_count_vec, y_train)
forest = RandomForestClassifier(n_estimators=100, random_state=0).fit(X_train_count_vec, y_train)

In [11]:
len(X_test)

308

In [12]:
X_test_unigram_count_vec = unigram_count_vectorizer.transform(X_test)

In [13]:
print(X_train_count_vec.shape)

(1228, 3040)


In [14]:
#1. MultinomialNB
y_pred = clf.predict(X_test_unigram_count_vec)
print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9740    0.9740    0.9740       154
           1     0.9740    0.9740    0.9740       154

    accuracy                         0.9740       308
   macro avg     0.9740    0.9740    0.9740       308
weighted avg     0.9740    0.9740    0.9740       308



In [15]:
#2 SVM
y_pred = svm.predict(X_test_unigram_count_vec)
print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8902    0.9481    0.9182       154
           1     0.9444    0.8831    0.9128       154

    accuracy                         0.9156       308
   macro avg     0.9173    0.9156    0.9155       308
weighted avg     0.9173    0.9156    0.9155       308



In [16]:
#3 Decision Tree
y_pred = tree.predict(X_test_unigram_count_vec)
print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8681    0.8117    0.8389       154
           1     0.8232    0.8766    0.8491       154

    accuracy                         0.8442       308
   macro avg     0.8456    0.8442    0.8440       308
weighted avg     0.8456    0.8442    0.8440       308



In [17]:
#4 Logistic Regression
y_pred = regression.predict(X_test_unigram_count_vec)
print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9119    0.9416    0.9265       154
           1     0.9396    0.9091    0.9241       154

    accuracy                         0.9253       308
   macro avg     0.9258    0.9253    0.9253       308
weighted avg     0.9258    0.9253    0.9253       308



In [18]:
#5 Random Forest
y_pred = forest.predict(X_test_unigram_count_vec)
print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9156    0.9156    0.9156       154
           1     0.9156    0.9156    0.9156       154

    accuracy                         0.9156       308
   macro avg     0.9156    0.9156    0.9156       308
weighted avg     0.9156    0.9156    0.9156       308

